In [2]:
import pandas as pd

# Importing the dataset
dataset = pd.read_csv("data/tracks_with_genres.csv")

# Creating the dataframe
df = pd.DataFrame(dataset)

In [ ]:
# Spotify API Authentication Information
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
#weilin's api key
client_id = '80e8701040fd4ee99393dbe0038161fb'
client_secret = 'e5b3e975af1a4ed8adac362bddbe98f3'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)